In [1]:
from webscrapping.database.sql_queries import ValorantQueries
import pandas as pd

In [2]:
vq = ValorantQueries()
vq.set_match(43621)

Connected to database valorant successfully
[('teams',), ('roundevents',), ('players',), ('events',), ('series',), ('matches',), ('maps',), ('rounds',), ('roundeconomies',), ('roundlocations',), ('playermapinstance',), ('assists',)]


In [3]:
vq.aggregate_match_gamestate()

,Timestamp,EventType,ATK_Loadout_Value,ATK_Spread,ATK_Shield,ATK_Weapon_Price,ATK_Initiator,Map_Name,ATK_Duelist,ATK_Controller,...,DEF_Spread,DEF_Loadout_Value,DEF_Shield,DEF_Weapon_Price,DEF_Initiator,DEF_Duelist,DEF_Controller,DEF_Sentinel,DEF_has_OP,FinalWinner
0,0,start,3950,0.000000,50,1500,1,Icebox,2,1,...,0.000000,3750,50,1800,2,1,1,1,0,1
1,17272,kill,3950,122.266135,50,1500,1,Icebox,2,1,...,315.665394,2950,50,1000,2,0,1,1,0,1
2,29339,kill,3950,160.990222,50,1500,1,Icebox,2,1,...,267.709079,2200,50,500,1,0,1,1,0,1
3,29760,kill,3950,165.174399,50,1500,1,Icebox,2,1,...,301.271910,1400,25,500,1,0,1,0,0,1
4,37694,plant,3950,229.418092,50,1500,1,Icebox,2,1,...,224.051269,1400,25,500,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,93788,plant,4750,315.346712,100,1900,0,Icebox,0,1,...,108.325930,16150,150,10800,1,1,0,1,1,0
7,94458,kill,2550,294.703493,50,1100,0,Icebox,0,0,...,116.540651,16150,150,10800,1,1,0,1,1,0
8,107910,kill,2550,141.243609,50,1100,0,Icebox,0,0,...,0.000000,11450,100,7900,1,1,0,0,1,0
9,109289,kill,0,121.361877,0,0,0,Icebox,0,0,...,0.000000,11450,100,7900,1,1,0,0,1,0


In [9]:
# Get score table where round_number == 1
score_table = vq.get_full_round()
score_table_df = pd.DataFrame(score_table, columns=["RoundID", "MatchID", "RoundNumber", "Attacking_Team", "WinningTeam", "Team_A_Economy", "Team_B_Economy", "Win_condition", "Ceremony"])
score_table_df = score_table_df[score_table_df["RoundNumber"] == 5]
attacking_team = int(score_table_df["Attacking_Team"])
winning_team = int(score_table_df["WinningTeam"])
winner = 1 if attacking_team == winning_team else 0
winner

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


In [6]:
vq.get_full_round()

1

In [3]:
score_table

[(695324, 43621, 1, 53, 53, 0, 0, 'kills', 'flawless'),
 (695325, 43621, 2, 53, 53, 3, 1, 'kills', 'team_ace'),
 (695326, 43621, 3, 53, 53, 3, 4, 'bomb', 'closer'),
 (695327, 43621, 4, 53, 53, 4, 3, 'bomb', 'default'),
 (695328, 43621, 5, 53, 53, 4, 4, 'kills', 'default'),
 (695329, 43621, 6, 53, 444, 4, 2, 'defuse', 'thrifty'),
 (695330, 43621, 7, 53, 444, 4, 4, 'defuse', 'default'),
 (695331, 43621, 8, 53, 444, 4, 4, 'defuse', 'default'),
 (695332, 43621, 9, 53, 53, 3, 4, 'kills', 'default'),
 (695333, 43621, 10, 53, 444, 4, 1, 'defuse', 'thrifty'),
 (695334, 43621, 11, 53, 444, 3, 4, 'defuse', 'default'),
 (695335, 43621, 12, 53, 444, 4, 4, 'time', 'default'),
 (695336, 43621, 13, 444, 444, 0, 0, 'kills', 'default'),
 (695337, 43621, 14, 444, 444, 1, 3, 'kills', 'default'),
 (695338, 43621, 15, 444, 53, 4, 3, 'kills', 'default'),
 (695339, 43621, 16, 444, 53, 4, 2, 'kills', 'flawless'),
 (695340, 43621, 17, 444, 53, 4, 4, 'time', 'default'),
 (695341, 43621, 18, 444, 53, 4, 3, 'defu